In [52]:
import pandas as pd
import numpy as np
import pickle as p

In [53]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=153, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [54]:
# item_subset

### `/train`

In [55]:
import time

In [56]:
import requests
import json
from threading import Thread

url = 'https://fillet.azurewebsites.net/train/'

data_subset.to_parquet('data_subset.parquet')
files = {'data': open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True, 'project_id': 2}

Thread(target=requests.post, args=(url, ), kwargs={'data': payload, 'files': files}).start()

time.sleep(10)
files['data'].close()

### `/query_progress`

In [72]:
import requests
import json

url = 'https://fillet.azurewebsites.net/query_progress/'


payload = {'project_id': 2}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=1200)

print('% Complete:',round(r.json()['pct_complete'],3))
print('CV Done:',r.json()['cv_done'])

% Complete: 100.0
CV Done: 0


In [48]:
set(r.json()['project_items']) - set(r.json()['train_complete'])

set()

### `/get_cv_results`

In [70]:
import requests
import json

url = 'https://fillet.azurewebsites.net/get_cv_results/'


payload = {'project_id': 2}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=1200)
pd.DataFrame().from_dict((json.loads(r.json())))

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [71]:
r.json()

{'status': 'incomplete'}

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [73]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
# a_input


# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'

payload = {'prices': a_input, 'project_id': 2}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

Input Prices Quantities:


,Estimated,Actual
Qty_1,512.0,520.0
Qty_1002,3190.0,3520.0
Qty_1003,3531.0,3624.0
Qty_1004,3821.0,3789.0
Qty_101,304.0,349.0
...,...,...
Qty_7434,304.0,236.0
Qty_7575,103.0,146.0
Qty_7965,50.0,59.0
Qty_7966,129.0,182.0
